In [89]:
import gensim
import re
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import seaborn as sns


pandarallel.initialize(progress_bar=True, nb_workers=6)
pd.options.display.max_rows = 200


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Sentence-level Tokenization


In [46]:
kdf = pd.read_csv("../v1/kawiki.csv")
kdf.text = kdf.text.fillna('')

kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False


In [59]:
def cleanup(doc):
    # doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ0-9-–—_%a-zA-Z]", ' ', doc)
    doc = str(doc).lower()
    doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ\.]", ' ', doc)
    doc = re.sub(r"\s+", ' ', doc)
    return doc


# kdf['cleantext'] = kdf['title'] + '. ' + kdf['text']
kdf['cleantext'] = kdf.text.parallel_apply(cleanup)
kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ ამბროსის ძე შევარდნაძე დ. იანვარი სოფე...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი გრიგორიანული კალენდრის პირველი დღე. წ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი გრიგორიანული კალენდრის ე დღე. წლის ბო...


In [60]:
# Drop useless wiki articles

# Empty articles
kdf.drop(kdf[(kdf.cleantext == ' ') | (kdf.cleantext == '')].index, inplace=True)
kdf.drop(kdf[kdf.cleantext.apply(lambda x: bool(re.compile("[ა-ჿ]").search(x)))==False].index, inplace=True)

# Mediawiki articles
kdf.drop(kdf[kdf.title.str.startswith("მედიავიკი:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("ვიკიპედია:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("თარგი:")].index, inplace=True)

# Redirection
kdf.drop(kdf[kdf.isRedirection].index, inplace=True)

kdf

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ ამბროსის ძე შევარდნაძე დ. იანვარი სოფე...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი გრიგორიანული კალენდრის პირველი დღე. წ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი გრიგორიანული კალენდრის ე დღე. წლის ბო...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ მო...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო სახელმწიფო ევრაზიაში კავკასიაში შა...
...,...,...,...,...,...,...,...,...
320738,532936,2022-05-19T20:24:07Z,კატეგორია:სამხრეთ კორეის საერთაშორისო ხელშეკრუ...,იაკობ მახარაძე,კატეგორია:საერთაშორისო ხელშეკრულებები ქვეყნები...,1mvdnl2ggxg80q12sh5aw4dwfbbt3pu,False,კატეგორია საერთაშორისო ხელშეკრულებები ქვეყნები...
320740,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l,False,ვახტანგ რობაქიძე მოსამართლეობის კანდიდატი დაბ
320741,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,\n\nნინა კარპაჩოვა (უკრ. Ні́на Іва́нівна Карпа...,m955153juz99zg6n3e4pphtna8s6f8t,False,ნინა კარპაჩოვა უკრ. დ. აგვისტო მოლდავეთის სსრ...
320743,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,\nthumb|right|250px|მემორიალური ნიშნული ოფიცერ...,6sgtfdyggb9svx8whe22owjhwkm8fia,False,მემორიალური ნიშნული ოფიცერი კიზევეტერის მკვლე...


In [61]:
kdf['cleantext'] = kdf['title'].apply(cleanup) + '. ' + kdf['cleantext']
kdf.head()

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ შევარდნაძე. ედუარდ ამბროსის ძე შევარდნ...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი. იანვარი გრიგორიანული კალენდრის პირვ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი. იანვარი გრიგორიანული კალენდრის ე დღ...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,. მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო. საქართველო სახელმწიფო ევრაზიაში კ...


In [64]:
fulltext = '.'.join(kdf.cleantext.values)
sentences = fulltext.split('.')
sentences = [sentence.split(' ') for sentence in tqdm(sentences)]

for idx, sentence in tqdm(enumerate(sentences), total=len(sentences)):
    sentences[idx] = [
        token for token in sentence 
        if token != '' 
        and len(token) > 1
    ]

sentences = [sentence for sentence in tqdm(sentences) if sentence != []]

sentences[:5]

100%|██████████| 3362533/3362533 [00:00<00:00, 6412523.25it/s]


[['ედუარდ', 'შევარდნაძე'],
 ['ედუარდ', 'ამბროსის', 'ძე', 'შევარდნაძე'],
 ['იანვარი', 'სოფელი', 'მამათი', 'ოზურგეთის', 'მაზრა', 'საქართველოს', 'სსრ'],
 ['ივლისი',
  'თბილისი',
  'საქართველო',
  'ქართველი',
  'პოლიტიკოსი',
  'და',
  'სახელმწიფო',
  'მოღვაწე'],
 ['საქართველოს', 'მეორე', 'პრეზიდენტი']]

In [74]:
wordfreq = defaultdict(int)

for sentence in tqdm(sentences):
    for token in sentence:
        wordfreq[token] += 1

print(f"Total words:\t\t\t{len(wordfreq)}")
print(f"Total words with count=1:\t{len(list(filter(lambda x : x[1] == 1, wordfreq.items())))}")

100%|██████████| 2754036/2754036 [00:06<00:00, 446409.97it/s]


Total words:			1079347
Total words with count=1:	571881


In [76]:
# Filter out spelling errors and extremely infrequent words that models are unlikely to capture any useful signals
tokenCorpus = [
    [token for token in sentence if wordfreq[token] > 1]
    for sentence in tqdm(sentences)
]

# Filter out single-word sentences
tokenCorpus = [sentence for sentence in tqdm(tokenCorpus) if len(sentence) > 1]

100%|██████████| 2754036/2754036 [00:00<00:00, 5939979.46it/s]


In [77]:
with open("kawiki_corpus_sentences.txt", "w") as file:
    for doc in tqdm(tokenCorpus):
        file.write(' '.join(doc) + '\n')

100%|██████████| 2562447/2562447 [00:02<00:00, 1000135.00it/s]


100%|██████████| 2611926/2611926 [00:02<00:00, 928658.19it/s]


# Article-level Tokenization

In [78]:
kdf = pd.read_csv("../v1/kawiki.csv")
kdf.text = kdf.text.fillna('')

kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False


In [79]:
def cleanup(doc):
    # doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ0-9-–—_%a-zA-Z]", ' ', doc)
    doc = str(doc).lower()
    doc = re.sub(r"[^Ⴀ-ჿⴀ-ⴥᲐ-Ჿ]", ' ', doc)
    doc = re.sub(r"\s+", ' ', doc)
    return doc


kdf['cleantext'] = kdf.text.parallel_apply(cleanup)
kdf.head(3)

,id,ts,title,author,text,sha1,isRedirection,cleantext
0,2,2004-01-26T08:48:14Z,ვიკიპედია:Orphaned articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",ptlg4dsza52o10rnmk59f282y0fptzi,False,
1,3,2004-01-26T00:26:51Z,ვიკიპედია:Most wanted articles,204.95.98.251,,pezx7385o3910tfwsxb3suac2dw5g9i,False,
2,4,2004-01-26T00:38:52Z,ვიკიპედია:Short articles,204.95.98.251,"<a href=""/wiki/Main_Page"" class='internal' tit...",84gitu5sgfaec85yfccjzvl2kfafvn1,False,


In [80]:
# Drop useless wiki articles

# Empty articles
kdf.drop(kdf[(kdf.cleantext == ' ') | (kdf.cleantext == '')].index, inplace=True)
kdf.drop(kdf[kdf.cleantext.apply(lambda x: bool(re.compile("[ა-ჿ]").search(x)))==False].index, inplace=True)

# Mediawiki articles
kdf.drop(kdf[kdf.title.str.startswith("მედიავიკი:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("ვიკიპედია:")].index, inplace=True)
kdf.drop(kdf[kdf.title.str.startswith("თარგი:")].index, inplace=True)

# Redirection
kdf.drop(kdf[kdf.isRedirection].index, inplace=True)

kdf

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ ამბროსის ძე შევარდნაძე დ იანვარი სოფელ...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი გრიგორიანული კალენდრის პირველი დღე წლ...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი გრიგორიანული კალენდრის ე დღე წლის ბოლ...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ მო...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო სახელმწიფო ევრაზიაში კავკასიაში შა...
...,...,...,...,...,...,...,...,...
320738,532936,2022-05-19T20:24:07Z,კატეგორია:სამხრეთ კორეის საერთაშორისო ხელშეკრუ...,იაკობ მახარაძე,კატეგორია:საერთაშორისო ხელშეკრულებები ქვეყნები...,1mvdnl2ggxg80q12sh5aw4dwfbbt3pu,False,კატეგორია საერთაშორისო ხელშეკრულებები ქვეყნები...
320740,532938,2022-05-20T04:35:44Z,ვახტანგ რობაქიძე,Jaba1977,ვახტანგ რობაქიძე (მოსამართლეობის კანდიდატი) და...,iavvdscaounfaz6sqowhc2ogsvp4u0l,False,ვახტანგ რობაქიძე მოსამართლეობის კანდიდატი დაბ
320741,532939,2022-05-20T08:54:31Z,ნინა კარპაჩოვა,Ekkatterrinna,\n\nნინა კარპაჩოვა (უკრ. Ні́на Іва́нівна Карпа...,m955153juz99zg6n3e4pphtna8s6f8t,False,ნინა კარპაჩოვა უკრ დ აგვისტო მოლდავეთის სსრ უ...
320743,532947,2022-05-20T08:27:24Z,ჰაილბრონის აჩრდილი,Lisztomaniac,\nthumb|right|250px|მემორიალური ნიშნული ოფიცერ...,6sgtfdyggb9svx8whe22owjhwkm8fia,False,მემორიალური ნიშნული ოფიცერი კიზევეტერის მკვლე...


In [81]:
kdf['cleantext'] = kdf['title'].apply(cleanup) + ' ' + kdf['cleantext'] + ' ' + kdf['title'].apply(cleanup)
kdf.head()

,id,ts,title,author,text,sha1,isRedirection,cleantext
234,604,2022-04-15T07:50:59Z,ედუარდ შევარდნაძე,Melberg,\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი...,k4tckhizmxggllhid4m0uob0nmwbu0h,False,ედუარდ შევარდნაძე ედუარდ ამბროსის ძე შევარდნა...
236,608,2020-12-01T07:15:40Z,1 იანვარი,Namdvili,\n1 იანვარი — გრიგორიანული კალენდრის პირველი დ...,9hua4ulox6vj429j2nbx3404fylhw5t,False,იანვარი იანვარი გრიგორიანული კალენდრის პირვე...
237,609,2020-01-18T07:59:48Z,25 იანვარი,95.104.15.233,\n25 იანვარი — გრიგორიანული კალენდრის 25-ე დღე...,nburza2tu9lqypz9cirawwpcg3s7e8z,False,იანვარი იანვარი გრიგორიანული კალენდრის ე დღე...
238,610,2021-11-08T19:32:05Z,1928,Arkaitz1974,\n== მოვლენები ==\n* 11 თებერვალი : შვეიცარიის...,g2uj2d76l6yrbtu1p4wvmnx7vacz64z,False,მოვლენები თებერვალი შვეიცარიის ქალაქ ზანქტ ...
282,823,2022-05-09T20:01:56Z,საქართველო,GiorgiXIII,"\n\nსაქართველო () — სახელმწიფო ევრაზიაში, კავკ...",9kmab2upb1nr0ad1ywgr6ldl787kyo3,False,საქართველო საქართველო სახელმწიფო ევრაზიაში კა...


In [91]:
articles = kdf.cleantext.values
articles = [sentence.split(' ') for sentence in tqdm(articles)]

for idx, sentence in tqdm(enumerate(articles), total=len(articles)):
    articles[idx] = [
        token for token in sentence 
        if token != '' 
        and len(token) > 1
    ]

articles = [sentence for sentence in tqdm(articles) if sentence != []]

articles[0][:10]

100%|██████████| 229121/229121 [00:00<00:00, 5186565.31it/s]


['ედუარდ',
 'შევარდნაძე',
 'ედუარდ',
 'ამბროსის',
 'ძე',
 'შევარდნაძე',
 'იანვარი',
 'სოფელი',
 'მამათი',
 'ოზურგეთის']

In [93]:
wordfreq = defaultdict(int)

for article in tqdm(articles):
    for token in article:
        wordfreq[token] += 1

print(f"Total words:\t\t\t{len(wordfreq)}")
print(f"Total words with count=1:\t{len(list(filter(lambda x : x[1] == 1, wordfreq.items())))}")

100%|██████████| 229121/229121 [00:06<00:00, 32737.60it/s]


Total words:			1079347
Total words with count=1:	570816


In [94]:
# Filter out spelling errors and extremely infrequent words that models are unlikely to capture any useful signals
tokenCorpus = [
    [token for token in article if wordfreq[token] > 1]
    for article in tqdm(articles)
]

# Filter out single-word articles
tokenCorpus = [article for article in tqdm(tokenCorpus) if len(article) > 1]

100%|██████████| 229121/229121 [00:00<00:00, 3471970.08it/s]


In [95]:
with open("kawiki_corpus_articles.txt", "w") as file:
    for doc in tqdm(tokenCorpus):
        file.write(' '.join(doc) + '\n')

100%|██████████| 229120/229120 [00:01<00:00, 137328.62it/s]
